In [1]:
import pandas as pd
from ast import literal_eval
from tqdm import tqdm
from abc import ABC, abstractmethod
from typing import List
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
tqdm.pandas()

# Content-based movies recommendations

## 1. Data loading and processing

### Ratings

In [3]:
ratings = pd.read_csv("../data/ratings_small.csv")

In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
ratings.columns = ['user_id', 'movie_id', 'rating', 'timestamp']

### Links

In [6]:
links = pd.read_csv("../data/links_small.csv")

In [7]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [8]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB


In [9]:
links.columns = ['movie_id', 'imbd_id', 'tmbd_id']
links=links.dropna()

In [10]:
links['tmbd_id'] = links['tmbd_id'].astype('int')
links['movie_id'] = links['movie_id'].astype('int')

In [11]:
links.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9112 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   movie_id  9112 non-null   int64
 1   imbd_id   9112 non-null   int64
 2   tmbd_id   9112 non-null   int64
dtypes: int64(3)
memory usage: 284.8 KB


### Movies

#### Metadata

In [12]:
metadata = pd.read_csv("../data/movies_metadata.csv")

/home/kornel/Projects/recomendation-systems/cb-venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [13]:
metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [14]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [15]:
# drop 3 incorrect rows
metadata = metadata.drop([19730, 29503, 35587])

# genres column
metadata['genres'] = metadata.progress_apply(lambda row: [x['name'].replace(' ', '_').lower() for x in literal_eval(row['genres'])], axis=1)

# collection column
metadata['belongs_to_collection'].fillna('{\'name\': \'\'}', inplace=True)
metadata['collection'] = metadata.progress_apply(lambda row: literal_eval(row['belongs_to_collection'])['name'], axis=1)

# movie decrisption
metadata['overview'].fillna('', inplace=True)
metadata['tagline'].fillna('', inplace=True)
metadata['description'] = metadata['overview'] + metadata['tagline']

# select interesting columns
col = ['id', 'title', 'collection', 'description', 'genres']
metadata = metadata[col]

100%|██████████| 45463/45463 [00:01<00:00, 37060.41it/s]


In [16]:
metadata.head()

,id,title,collection,description,genres
0,862,Toy Story,Toy Story Collection,"Led by Woody, Andy's toys live happily in his ...","[animation, comedy, family]"
1,8844,Jumanji,,When siblings Judy and Peter discover an encha...,"[adventure, fantasy, family]"
2,15602,Grumpier Old Men,Grumpy Old Men Collection,A family wedding reignites the ancient feud be...,"[romance, comedy]"
3,31357,Waiting to Exhale,,"Cheated on, mistreated and stepped on, the wom...","[comedy, drama, romance]"
4,11862,Father of the Bride Part II,Father of the Bride Collection,Just when George Banks has recovered from his ...,[comedy]


In [17]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45463 entries, 0 to 45465
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           45463 non-null  object
 1   title        45460 non-null  object
 2   collection   45463 non-null  object
 3   description  45463 non-null  object
 4   genres       45463 non-null  object
dtypes: object(5)
memory usage: 2.1+ MB


#### credits

In [18]:
creds = pd.read_csv('./../data/credits.csv')

In [19]:
creds.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [20]:
creds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [21]:
creds['cast'] = creds.progress_apply(lambda row: [x['name'].replace(' ', '_').lower() for x in literal_eval(row['cast'])[0:10]], axis=1)
creds['crew'] = creds.progress_apply(lambda row: [x['name'].replace(' ', '_').lower() for x in literal_eval(row['crew'])[0:10]], axis=1)

100%|██████████| 45476/45476 [00:15<00:00, 2862.77it/s]


In [22]:
creds.head()

,cast,crew,id
0,"[tom_hanks, tim_allen, don_rickles, jim_varney...","[john_lasseter, joss_whedon, andrew_stanton, j...",862
1,"[robin_williams, jonathan_hyde, kirsten_dunst,...","[larry_j._franco, jonathan_hensleigh, james_ho...",8844
2,"[walter_matthau, jack_lemmon, ann-margret, sop...","[howard_deutch, mark_steven_johnson, mark_stev...",15602
3,"[whitney_houston, angela_bassett, loretta_devi...","[forest_whitaker, ronald_bass, ronald_bass, ez...",31357
4,"[steve_martin, diane_keaton, martin_short, kim...","[alan_silvestri, elliot_davis, nancy_meyers, n...",11862


#### keywords

In [23]:
keywords = pd.read_csv('../data/keywords.csv')

In [24]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [25]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [26]:
keywords['keywords'] = keywords.progress_apply(lambda row: [x['name'].replace(' ', '_').lower() for x in literal_eval(row['keywords'])[0:10]], axis=1)

100%|██████████| 46419/46419 [00:02<00:00, 16627.29it/s]


In [27]:
keywords.head()

,id,keywords
0,862,"[jealousy, toy, boy, friendship, friends, riva..."
1,8844,"[board_game, disappearance, based_on_children'..."
2,15602,"[fishing, best_friend, duringcreditsstinger, o..."
3,31357,"[based_on_novel, interracial_relationship, sin..."
4,11862,"[baby, midlife_crisis, confidence, aging, daug..."


#### Merge data

In [28]:
metadata['id'] = metadata['id'].astype('float64')
movies = pd.merge(metadata, creds, on='id')
movies = pd.merge(movies, keywords, on='id').dropna()

In [29]:
columns = ['id', 'title', 'collection', 'description', 'genres', 'keywords', 'cast', 'crew']
movies = movies[columns]

In [30]:
movies.head()

,id,title,collection,description,genres,keywords,cast,crew
0,862.0,Toy Story,Toy Story Collection,"Led by Woody, Andy's toys live happily in his ...","[animation, comedy, family]","[jealousy, toy, boy, friendship, friends, riva...","[tom_hanks, tim_allen, don_rickles, jim_varney...","[john_lasseter, joss_whedon, andrew_stanton, j..."
1,8844.0,Jumanji,,When siblings Judy and Peter discover an encha...,"[adventure, fantasy, family]","[board_game, disappearance, based_on_children'...","[robin_williams, jonathan_hyde, kirsten_dunst,...","[larry_j._franco, jonathan_hensleigh, james_ho..."
2,15602.0,Grumpier Old Men,Grumpy Old Men Collection,A family wedding reignites the ancient feud be...,"[romance, comedy]","[fishing, best_friend, duringcreditsstinger, o...","[walter_matthau, jack_lemmon, ann-margret, sop...","[howard_deutch, mark_steven_johnson, mark_stev..."
3,31357.0,Waiting to Exhale,,"Cheated on, mistreated and stepped on, the wom...","[comedy, drama, romance]","[based_on_novel, interracial_relationship, sin...","[whitney_houston, angela_bassett, loretta_devi...","[forest_whitaker, ronald_bass, ronald_bass, ez..."
4,11862.0,Father of the Bride Part II,Father of the Bride Collection,Just when George Banks has recovered from his ...,[comedy],"[baby, midlife_crisis, confidence, aging, daug...","[steve_martin, diane_keaton, martin_short, kim...","[alan_silvestri, elliot_davis, nancy_meyers, n..."


In [31]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46624 entries, 0 to 46627
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           46624 non-null  float64
 1   title        46624 non-null  object 
 2   collection   46624 non-null  object 
 3   description  46624 non-null  object 
 4   genres       46624 non-null  object 
 5   keywords     46624 non-null  object 
 6   cast         46624 non-null  object 
 7   crew         46624 non-null  object 
dtypes: float64(1), object(7)
memory usage: 3.2+ MB


#### Transform id to that used in ratings csv. Drop movies not rated in ratings_small. Drop row duplicated by movie_id.

In [32]:
movies = movies.rename(columns={'id': 'tmbd_id'})
movies['tmbd_id'] = movies['tmbd_id'].astype('int')
movies = pd.merge(links, movies, on='tmbd_id')
movies = movies.drop(columns=['imbd_id', 'tmbd_id'])

In [33]:
movies = movies[movies['movie_id'].isin(links['movie_id'])]

In [34]:
movies = movies.drop_duplicates('movie_id')

In [35]:
movies.head()

,movie_id,title,collection,description,genres,keywords,cast,crew
0,1,Toy Story,Toy Story Collection,"Led by Woody, Andy's toys live happily in his ...","[animation, comedy, family]","[jealousy, toy, boy, friendship, friends, riva...","[tom_hanks, tim_allen, don_rickles, jim_varney...","[john_lasseter, joss_whedon, andrew_stanton, j..."
1,2,Jumanji,,When siblings Judy and Peter discover an encha...,"[adventure, fantasy, family]","[board_game, disappearance, based_on_children'...","[robin_williams, jonathan_hyde, kirsten_dunst,...","[larry_j._franco, jonathan_hensleigh, james_ho..."
2,3,Grumpier Old Men,Grumpy Old Men Collection,A family wedding reignites the ancient feud be...,"[romance, comedy]","[fishing, best_friend, duringcreditsstinger, o...","[walter_matthau, jack_lemmon, ann-margret, sop...","[howard_deutch, mark_steven_johnson, mark_stev..."
3,4,Waiting to Exhale,,"Cheated on, mistreated and stepped on, the wom...","[comedy, drama, romance]","[based_on_novel, interracial_relationship, sin...","[whitney_houston, angela_bassett, loretta_devi...","[forest_whitaker, ronald_bass, ronald_bass, ez..."
4,5,Father of the Bride Part II,Father of the Bride Collection,Just when George Banks has recovered from his ...,[comedy],"[baby, midlife_crisis, confidence, aging, daug...","[steve_martin, diane_keaton, martin_short, kim...","[alan_silvestri, elliot_davis, nancy_meyers, n..."


In [36]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9082 entries, 0 to 9218
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movie_id     9082 non-null   int64 
 1   title        9082 non-null   object
 2   collection   9082 non-null   object
 3   description  9082 non-null   object
 4   genres       9082 non-null   object
 5   keywords     9082 non-null   object
 6   cast         9082 non-null   object
 7   crew         9082 non-null   object
dtypes: int64(1), object(7)
memory usage: 638.6+ KB


#### Helper dicts

In [37]:
title_by_df_id = {}
# movie_id_by_df_id = {}
df_id_by_movie_id = {}

for i in tqdm(range(len(movies))):
    title = movies.iloc[i]['title']
    movie_id = movies.iloc[i]['movie_id']
    title_by_df_id[i] = title
    # movie_id_by_df_id[i] = movie_id
    df_id_by_movie_id[movie_id] = i

100%|██████████| 9082/9082 [00:02<00:00, 3619.08it/s]


## 2. Recomendation framework

In [56]:
class BinaryRecommender():

    def __init__(
        self,
        movies_data: pd.DataFrame = movies
    ):
        self.movies = movies_data
        self.cosine_sim_matrix = None
        # cosine_sim_matrix to implement in sub classes

    def recommend(
        self,
        movie_id: float,
        n: int = 10
    ) -> List[float]:

        df_id = df_id_by_movie_id[movie_id]
        sim_row = self.cosine_sim_matrix[df_id]
        top_df_ids = np.argsort(sim_row)[::-1][1:n+1]
        # top_titles = list(map(lambda x: title_by_df_id[x], top_df_ids))

        return top_df_ids

    def validate(
        self
    ):
        print('Recommendations for Toy Story:')
        recommendations = self.recommend(1)
        display(self.movies.iloc[recommendations])
        print()
        print()
        print('Recommendations for Avatar:')
        recommendations = self.recommend(72998)
        display(self.movies.iloc[recommendations])

## 3. Recommendation systems

### 3.1. Description-based reccomendation

In [57]:
class DescriptionBasedRecommender(BinaryRecommender):
    # Description based movie recommender

    def __init__(
        self,
        vectorizer,
        movies_data: pd.DataFrame = movies
    ):
        super().__init__(movies_data)
        
        vec = vectorizer()
        features_matrix = vec.fit_transform(self.movies['description'])
        self.cosine_sim_matrix = cosine_similarity(features_matrix)

In [58]:
cbr = DescriptionBasedRecommender(CountVectorizer)
cbr.validate()

Recommendations for Toy Story:


,movie_id,title,collection,description,genres,keywords,cast,crew
2522,3114,Toy Story 2,Toy Story Collection,"Andy heads off to Cowboy Camp, leaving his toy...","[animation, comedy, family]","[museum, prosecution, identity_crisis, airplan...","[tom_hanks, tim_allen, joan_cusack, kelsey_gra...","[andrew_stanton, andrew_stanton, lee_unkrich, ..."
7621,78499,Toy Story 3,Toy Story Collection,"Woody, Buzz, and the rest of Andy's toys haven...","[animation, family, comedy]","[hostage, college, toy, barbie, animation, esc...","[tom_hanks, tim_allen, ned_beatty, joan_cusack...","[andrew_stanton, lee_unkrich, lee_unkrich, joh..."
5407,8199,Ugetsu,,"In the civil wars of 16th century Japan, two a...","[fantasy, drama, mystery]","[japan, based_on_novel, samurai, greed, bigamy...","[masayuki_mori, machiko_kyô, kinuyo_tanaka, ei...","[masaichi_nagata, fumio_hayasaka, kazuo_miyaga..."
4306,5690,Grave of the Fireflies,,"In the latter part of World War II, a boy and ...","[animation, drama, war]","[japan, brother_sister_relationship, hunger, w...","[tsutomu_tatsumi, ayano_shiraishi, yoshiko_shi...","[isao_takahata, isao_takahata, takeshi_seyama,..."
7170,64620,Frost/Nixon,,For three years after being forced from office...,"[drama, history]","[camera, lie, scandal, watergate_scandal, rich...","[michael_sheen, frank_langella, kevin_bacon, s...","[hans_zimmer, brian_grazer, tim_bevan, eric_fe..."
5788,26122,Onibaba,,"In medieval times, on a remote plain, a widow ...",[horror],"[women, jealousy, samurai, deserter, hold-up_r...","[nobuko_otowa, jitsuko_yoshimura, kei_satô, ju...","[kaneto_shindô, kaneto_shindô, kaneto_shindô, ..."
1960,2434,Down in the Delta,,Rosa Lynn sends her druggie daughter Loretta a...,[drama],[woman_director],"[alfre_woodard, al_freeman,_jr., esther_rolle,...","[maya_angelou, myron_goble]"
6080,31410,Downfall,,"In April of 1945, Germany stands at the brink ...","[drama, history, war]","[berlin, poison, dictator, clerk, ideology, wo...","[bruno_ganz, alexandra_maria_lara, corinna_har...","[oskar_roehler, bernd_eichinger, bernd_eiching..."
6034,30749,Hotel Rwanda,,"Inspired by true events, this film takes place...","[drama, history, war]","[rwanda, refugee, refugee_camp, militia, murde...","[don_cheadle, sophie_okonedo, nick_nolte, fana...","[nicholas_meyer, terry_george, terry_george, t..."
7221,66246,Uuno Turhapuro,Uuno Turhapuro,Uuno goes with his wife to spend summer holida...,[comedy],[],"[vesa-matti_loiri, marjatta_raita, spede_pasan...","[ere_kokkonen, spede_pasanen]"




Recommendations for Avatar:


,movie_id,title,collection,description,genres,keywords,cast,crew
6955,58156,Semi-Pro,,"Jackie Moon is the owner, promoter, coach, and...",[comedy],"[sport, basketball, flint_michigan, nba, merge...","[will_ferrell, maura_tierney, woody_harrelson,...","[scot_armstrong, kent_alterman, brian_avery]"
7195,65261,Ponyo,,"The son of a sailor, 5-year old Sosuke lives a...","[animation, family]","[mother, fish, cliff, father, mermaid, princes...","[yuria_nara, hiroki_doi, george_tokoro, tomoko...","[hayao_miyazaki, hayao_miyazaki, hayao_miyazak..."
6915,56801,Aliens vs Predator: Requiem,AVP Collection,"A sequel to 2004's Alien vs. Predator, the ico...","[fantasy, action, science_fiction, thriller, h...","[predator, national_guard, hybrid, alien, morg...","[steven_pasquale, reiko_aylesworth, john_ortiz...","[david_giler, jim_thomas, john_thomas, walter_..."
3457,4337,The Sand Pebbles,,Engineer Jake Holman arrives aboard the gunboa...,"[action, adventure, drama, war, romance]","[missionary, china, u.s._navy, chinese, battle...","[steve_mcqueen, richard_attenborough, richard_...","[robert_wise, robert_wise, boris_leven, jerry_..."
8739,114028,Pride,,Based on a trues story. In the summer of 1984 ...,"[drama, comedy]",[lgbt],"[ben_schnetzer, bill_nighy, imelda_staunton, d...","[matthew_warchus, stephen_beresford, david_liv..."
4849,6787,All the President's Men,,"In the run-up to the 1972 elections, Washingto...","[drama, history, mystery, thriller]","[usa, journalist, newspaper, burglar, fbi, wir...","[robert_redford, dustin_hoffman, jack_warden, ...","[william_goldman, gordon_willis, alan_j._pakul..."
5419,8257,Battleground,,The film concentrates on the camaraderie and t...,"[action, adventure, drama, war]",[world_war_ii],"[van_johnson, john_hodiak, ricardo_montalban, ...","[cedric_gibbons, john_d._dunning, william_a._w..."
7058,60609,Death Note,Death Note Collection,"Light Yagami finds the ""Death Note,"" a noteboo...","[fantasy, mystery, thriller]",[manga],"[tatsuya_fujiwara, kenichi_matsuyama, sota_aoy...","[shusuke_kaneko, takeshi_obata, tetsuya_oishi,..."
5771,26007,The Unknown Soldier,,It is the summer of 1941. An eastern-Finnish m...,"[drama, war]","[world_war_ii, trenches, trench_warfare]","[kosti_klemelä, heikki_savolainen, reino_tolva...","[t.j._särkkä, väinö_linna, edvin_laine, juha_n..."
546,620,Scream of Stone,,"A meeting of two world famous climbers, one an...","[adventure, drama, documentary]","[mountain, mountaineer, climbing]","[donald_sutherland, brad_dourif, vittorio_mezz...","[werner_herzog, robert_geoffrion, walter_saxer..."


In [59]:
cbr = DescriptionBasedRecommender(TfidfVectorizer)
cbr.validate()

Recommendations for Toy Story:


,movie_id,title,collection,description,genres,keywords,cast,crew
7621,78499,Toy Story 3,Toy Story Collection,"Woody, Buzz, and the rest of Andy's toys haven...","[animation, family, comedy]","[hostage, college, toy, barbie, animation, esc...","[tom_hanks, tim_allen, ned_beatty, joan_cusack...","[andrew_stanton, lee_unkrich, lee_unkrich, joh..."
2522,3114,Toy Story 2,Toy Story Collection,"Andy heads off to Cowboy Camp, leaving his toy...","[animation, comedy, family]","[museum, prosecution, identity_crisis, airplan...","[tom_hanks, tim_allen, joan_cusack, kelsey_gra...","[andrew_stanton, andrew_stanton, lee_unkrich, ..."
6259,35836,The 40 Year Old Virgin,,Andy Stitzer has a pleasant life with a nice a...,"[comedy, romance]","[first_time, virgin]","[steve_carell, catherine_keener, paul_rudd, ro...","[steve_carell, zachary_woodlee, judd_apatow, j..."
2567,3174,Man on the Moon,,A film about the life and career of the eccent...,"[comedy, drama, romance]","[show_business, comedian, wrestling]","[jim_carrey, courtney_love, bob_zmuda, danny_d...","[miloš_forman, scott_alexander, larry_karaszew..."
910,1103,Rebel Without a Cause,,"After moving to a new town, troublemaking teen...",[drama],"[individual, rebel, street_gang, car_race, par...","[james_dean, natalie_wood, sal_mineo, ann_dora...","[nicholas_ray, david_weisbart, nicholas_ray, i..."
4749,6584,"What's Up, Tiger Lily?",,"In comic Woody Allen's film debut, he took the...","[action, comedy]",[spoof],"[tatsuya_mihashi, akiko_wakabayashi, mie_hama,...","[woody_allen, senkichi_taniguchi]"
6700,51174,Factory Girl,,"In the mid-1960s, wealthy debutant Edie Sedgwi...",[drama],"[new_york, alcohol, sex, infidelity, nudity, a...","[sienna_miller, guy_pearce, hayden_christensen...","[ed_shearmur, john_dunn, george_hickenlooper, ..."
6627,49220,For Your Consideration,,Three actors learn that their respective perfo...,"[comedy, drama]",[],"[catherine_o'hara, harry_shearer, parker_posey...","[christopher_guest, christopher_guest, eugene_..."
1015,1244,Manhattan,,The life of a divorced television writer datin...,"[comedy, drama, romance]","[new_york, adultery, based_on_novel, lolita, l...","[woody_allen, diane_keaton, michael_murphy, ma...","[woody_allen, woody_allen, marshall_brickman, ..."
1619,2041,Condorman,,Comic artist and writer Woody performs a simpl...,"[comedy, action, adventure]",[mission],"[michael_crawford, oliver_reed, barbara_carrer...","[charles_jarrott, marc_stirdivant, henry_manci..."




Recommendations for Avatar:


,movie_id,title,collection,description,genres,keywords,cast,crew
2930,3657,Pandora and the Flying Dutchman,,Albert Lewin's interpretation of the legend of...,"[fantasy, drama, mystery, romance]","[legend, singer, bullfighter, flying_dutchman,...","[james_mason, ava_gardner, nigel_patrick, shei...","[albert_lewin, joe_kaufmann, albert_lewin, ala..."
4742,6564,Lara Croft Tomb Raider: The Cradle of Life,Tomb Raider Collection,Lara Croft ventures to an underwater temple in...,"[action, adventure, fantasy, thriller]","[riddle, treasure, medallion, kenia, alexander...","[angelina_jolie, gerard_butler, noah_taylor, d...","[jan_de_bont, dean_georgaris, lawrence_gordon,..."
4553,6213,Tears of the Sun,,Navy SEAL Lieutenant A.K. Waters and his elite...,"[action, drama, war]","[u.s._army, nigeria, president]","[bruce_willis, monica_bellucci, cole_hauser, e...","[naomi_shohan, david_lazan, hans_zimmer, lisa_..."
7673,80126,The American,,Dispatched to a small Italian town to await fu...,"[crime, drama, thriller]","[prostitute, sweden, suspense, priest, very_li...","[george_clooney, violante_placido, thekla_reut...","[george_clooney, herbert_grönemeyer, enzo_sist..."
2079,2571,The Matrix,The Matrix Collection,"Set in the 22nd century, The Matrix tells the ...","[action, science_fiction]","[saving_the_world, artificial_intelligence, ma...","[keanu_reeves, laurence_fishburne, carrie-anne...","[joel_silver, don_davis, bill_pope, zach_staen..."
3380,4225,The Dish,,The true story of how the Parkes Radio Telesco...,"[history, comedy, drama]","[nasa, australia, moon_landing, independent_fi...","[sam_neill, patrick_warburton, kevin_harringto...","[rob_sitch, zlad!, tom_gleisner, jane_kennedy,..."
5091,7196,The Men,,Fred Zinnemann's sensitive film on the plight ...,[drama],"[nurse, war_veteran, wheelchair, paralysis, sw...","[marlon_brando, teresa_wright, everett_sloane,...","[edward_g._boyle, fred_zinnemann, carl_foreman..."
996,1223,A Grand Day Out,Wallace & Gromit Collection,Wallace and Gromit have run out of cheese and ...,"[adventure, animation, comedy, science_fiction...","[moon, inventor, missile, cheese, robot, dog, ...",[peter_sallis],"[nick_park, nick_park, rob_copeland, julian_no..."
7547,74795,Green Zone,,During the U.S.-led occupation of Baghdad in 2...,"[war, action, adventure, drama, thriller]","[weapon_of_mass_destruction, baghdad, iraqi ]","[matt_damon, greg_kinnear, brendan_gleeson, am...","[john_hubbard, lloyd_levin, amanda_mackey, tim..."
538,611,Hellraiser: Bloodline,Hellraiser Collection,"In the 22nd century, a scientist attempts to r...","[horror, thriller, science_fiction]","[artificial_intelligence, engineer, hell, pinh...","[bruce_ramsay, valentina_vargas, charlotte_cha...","[paul_rich, nancy_rae_stone, gerry_lively, jim..."


### 3.2. Genres and keywords based recommendation

In [60]:
class GenresAndKeywordsBasedRecommender(BinaryRecommender):
    # Recommender based on genres and keywords

    def __init__(
        self,
        vectorizer,
        movies_data: pd.DataFrame = movies
    ):
        super().__init__(movies_data)
        
        vec = vectorizer()
        features_matrix = vec.fit_transform(
            (self.movies['genres'] + self.movies['keywords']).apply(lambda x: ' '.join(x))
        )
        self.cosine_sim_matrix = cosine_similarity(features_matrix)

In [61]:
cbr = GenresAndKeywordsBasedRecommender(CountVectorizer)
cbr.validate()

Recommendations for Toy Story:


,movie_id,title,collection,description,genres,keywords,cast,crew
1432,1822,Meet the Deedles,,Two surfers end up as Yellowstone park rangers...,"[animation, comedy, family]",[],"[paul_walker, john_ashton, dennis_hopper, eric...","[steve_boyum, jim_herzfeld, steve_bartek, davi..."
2751,3429,Creature Comforts,,Interviews with the public are used to put wor...,"[animation, comedy, family]",[animation],[],[nick_park]
7621,78499,Toy Story 3,Toy Story Collection,"Woody, Buzz, and the rest of Andy's toys haven...","[animation, family, comedy]","[hostage, college, toy, barbie, animation, esc...","[tom_hanks, tim_allen, ned_beatty, joan_cusack...","[andrew_stanton, lee_unkrich, lee_unkrich, joh..."
4186,5480,Stuart Little 2,Stuart Little Collection,"Stuart, an adorable white mouse, still lives h...","[family, adventure, animation, comedy]","[mouse, falcon, bird, friendship, family]","[michael_j._fox, geena_davis, hugh_laurie, jon...","[alan_silvestri, douglas_wick, priscilla_nedd-..."
5279,7790,"Bon Voyage, Charlie Brown (and Don't Come Back!)",,"Charlie Brown, Linus, Peppermint Patty and Mar...","[animation, comedy, family]",[peanuts],"[arrin_skelley, laura_planting, casey_carlson,...","[phil_roman, bill_melendez, charles_m._schulz]"
1199,1460,That Darn Cat,,While making his nightly rounds in the neighbo...,"[comedy, family]",[],"[christina_ricci, doug_e._doug, dean_jones, pe...",[bob_spiers]
220,248,Houseguest,,"In hot water with the mob over an unpaid debt,...","[comedy, family]",[],"[sinbad, phil_hartman, jeffrey_jones, kim_grei...",[randall_miller]
5953,27369,Is It Fall Yet?,,Is It Fall Yet? continues the series' running ...,"[animation, comedy]",[],"[tracy_grandstaff, wendy_hoopes]","[karen_disher, peggy_nicoll, guy_moore, glenn_..."
7328,69685,Is It College Yet?,,"All vile things must come to an end, and for D...","[animation, comedy]",[],"[tracy_grandstaff, wendy_hoopes]",[karen_disher]
433,486,Life With Mikey,,Michael Chapman was once a child TV star. But ...,"[comedy, family]",[],"[michael_j._fox, christina_vidal, nathan_lane,...","[james_lapine, william_ivey_long, marc_lawrence]"




Recommendations for Avatar:


,movie_id,title,collection,description,genres,keywords,cast,crew
4009,5182,Hawk the Slayer,,"Hawk the Slayer, after seeing both his father ...","[fantasy, action, adventure, science_fiction]",[],"[jack_palance, john_terry, bernard_bresslaw, r...","[terry_marcel, terry_marcel]"
2730,3401,Baby: Secret of the Lost Legend,,Paleontologist and her husband discover a moth...,"[adventure, fantasy, action, science_fiction, ...",[],"[william_katt, sean_young, patrick_mcgoohan, j...","[bill_l._norton, ellen_greene, clifford_green,..."
4004,5171,The Time Machine,,"Hoping to alter the events of the past, a 19th...","[science_fiction, adventure, action]","[future, time_machine]","[guy_pearce, mark_addy, phyllida_law, sienna_g...","[klaus_badelt, david_valdes, john_logan, donal..."
7069,60937,The Mummy: Tomb of the Dragon Emperor,The Mummy Collection,"Archaeologist Rick O'Connell travels to China,...","[adventure, action, fantasy]",[],"[brendan_fraser, jet_li, john_hannah, maria_be...","[ronna_kress, stephen_sommers, sean_daniel, ro..."
8724,113345,Jupiter Ascending,,In a universe where human genetic material is ...,"[science_fiction, fantasy, action, adventure]","[jupiter, space, woman_director, 3d, interspec...","[mila_kunis, channing_tatum, sean_bean, eddie_...","[bruce_berman, alexander_berner, john_toll, lo..."
8377,102445,Star Trek Into Darkness,Star Trek: Alternate Reality Collection,When the crew of the Enterprise is called back...,"[action, adventure, science_fiction]","[spacecraft, friendship, sequel, futuristic, s...","[chris_pine, zachary_quinto, zoe_saldana, karl...","[j.j._abrams, bryan_burk, alex_kurtzman, damon..."
6316,39408,Left Behind III: World at War,Left Behind Collection,A year and a half ago the world was hit with t...,"[action, adventure, drama, fantasy, science_fi...",[],"[louis_gossett,_jr., kirk_cameron, brad_johnso...",[craig_r._baxley]
7697,80748,Alice in Wonderland,,"On a boring winter afternoon, Alice dreams, th...","[action, adventure, family, fantasy, science_f...",[alice_in_wonderland],"[charlotte_henry, richard_arlen, roscoe_ates, ...","[dimitri_tiomkin, joseph_l._mankiewicz, willia..."
2969,3699,Starman,,An alien takes the form of a young widow's hus...,"[action, adventure, romance, science_fiction]","[places_and_planets, widow, lovers, alien, pol...","[jeff_bridges, karen_allen, charles_martin_smi...","[john_carpenter, michael_douglas, jack_nitzsch..."
5284,7810,Babylon 5: A Call to Arms,Babylon 5 Collection,Allies of the Shadows seek revenge against hum...,"[action, drama, science_fiction, adventure]","[space_marine, nightmare, space_war, alien_lif...","[bruce_boxleitner, jerry_doyle, jeff_conaway, ...","[harry_cohen, juan_peralta, michael_vejar, j._..."


In [62]:
cbr = GenresAndKeywordsBasedRecommender(TfidfVectorizer)
cbr.validate()

Recommendations for Toy Story:


,movie_id,title,collection,description,genres,keywords,cast,crew
7621,78499,Toy Story 3,Toy Story Collection,"Woody, Buzz, and the rest of Andy's toys haven...","[animation, family, comedy]","[hostage, college, toy, barbie, animation, esc...","[tom_hanks, tim_allen, ned_beatty, joan_cusack...","[andrew_stanton, lee_unkrich, lee_unkrich, joh..."
1432,1822,Meet the Deedles,,Two surfers end up as Yellowstone park rangers...,"[animation, comedy, family]",[],"[paul_walker, john_ashton, dennis_hopper, eric...","[steve_boyum, jim_herzfeld, steve_bartek, davi..."
8143,95441,Ted,Ted Collection,"John Bennett, a man whose childhood wish of br...","[comedy, fantasy]","[friendship, love, teddy_bear, toy_comes_to_li...","[mark_wahlberg, mila_kunis, seth_macfarlane, j...","[sheila_jaffe, jeff_freeman, debra_mcguire, mi..."
3742,4749,3 Ninjas Kick Back,3 Ninjas Collection,"During a championship baseball match, the thre...","[adventure, action, comedy, family]","[underdog, hero, friendship, treasure_hunt, fr...","[victor_wong, max_elliott_slade, sean_fox, j._...","[charles_t._kanganis, mark_saltzman]"
2751,3429,Creature Comforts,,Interviews with the public are used to put wor...,"[animation, comedy, family]",[animation],[],[nick_park]
4168,5448,Hey Arnold! The Movie,,When a powerful developer named Mr. Scheck wan...,"[animation, family]",[],"[spencer_klein, francesca_smith, jamil_walker_...","[tuck_tucker, clint_bond_jr., craig_bartlett, ..."
848,1026,So Dear to My Heart,,The tale of Jeremiah Kincaid and his quest to ...,"[animation, drama, family]",[],"[bobby_driscoll, luana_patten, beulah_bondi, b...","[ted_sears, harold_d._schuster, hamilton_luske..."
6125,32153,Once Upon a Forest,,"A young mouse, mole and hedgehog risk their li...","[animation, family, adventure]","[friendship, animal]","[michael_crawford, ben_vereen, ellen_blain, be...","[charles_grosvenor, rae_lambert, al_holter, ri..."
7139,63479,Sex Drive,,A high school senior drives cross-country with...,"[comedy, adventure, romance]","[sex, jealousy, virgin, nudity, community, fri...","[josh_zuckerman, amanda_crew, clark_duke, jame...","[lisa_beach, sarah_katzman, george_folsey_jr.,..."
7991,90866,Hugo,,Hugo is an orphan boy living in the walls of a...,"[adventure, drama, family]","[library, clock, film_director, key, toy, boy,...","[ben_kingsley, sacha_baron_cohen, asa_butterfi...","[johnny_depp, robert_richardson, howard_shore,..."




Recommendations for Avatar:


,movie_id,title,collection,description,genres,keywords,cast,crew
5284,7810,Babylon 5: A Call to Arms,Babylon 5 Collection,Allies of the Shadows seek revenge against hum...,"[action, drama, science_fiction, adventure]","[space_marine, nightmare, space_war, alien_lif...","[bruce_boxleitner, jerry_doyle, jeff_conaway, ...","[harry_cohen, juan_peralta, michael_vejar, j._..."
3038,3780,Rocketship X-M,,Astronauts blast off to explore the moon on Ro...,[science_fiction],[space_travel],"[lloyd_bridges, osa_massen, john_emery, noah_b...","[kurt_neumann, orville_h._hampton, murray_lern..."
8377,102445,Star Trek Into Darkness,Star Trek: Alternate Reality Collection,When the crew of the Enterprise is called back...,"[action, adventure, science_fiction]","[spacecraft, friendship, sequel, futuristic, s...","[chris_pine, zachary_quinto, zoe_saldana, karl...","[j.j._abrams, bryan_burk, alex_kurtzman, damon..."
9195,159972,Approaching the Unknown,,Captain William Stanaforth is on a one-way sol...,"[thriller, drama, science_fiction]","[mars, space_travel, space_mission, space]","[mark_strong, luke_wilson, sanaa_lathan, ander...","[mark_elijah_rosenberg, mark_elijah_rosenberg,..."
4009,5182,Hawk the Slayer,,"Hawk the Slayer, after seeing both his father ...","[fantasy, action, adventure, science_fiction]",[],"[jack_palance, john_terry, bernard_bresslaw, r...","[terry_marcel, terry_marcel]"
8500,106002,Ender's Game,,Based on the classic novel by Orson Scott Card...,"[science_fiction, action, adventure]","[based_on_novel, intolerance, chosen_one, chil...","[asa_butterfield, harrison_ford, hailee_steinf...","[john_papsidera, donald_mcalpine, lee_smith, z..."
2281,2827,The Astronaut's Wife,,When astronaut Spencer Armacost returns to Ear...,"[drama, science_fiction, thriller]","[wife_husband_relationship, space_travel, spac...","[johnny_depp, charlize_theron, joe_morton, cle...","[richard_pagano, isis_mussenden, leslie_a._pop..."
7006,59392,Stargate: The Ark of Truth,Stargate SG-1 Collection,SG-1 searches for an ancient weapon which coul...,"[adventure, science_fiction]","[wormhole, space_travel, supernatural_powers, ...","[ben_browder, amanda_tapping, christopher_judg...","[joel_goldsmith, peter_woeste, james_bamford, ..."
1082,1320,Alien³,Alien Collection,After escaping with Newt and Hicks from the al...,"[science_fiction, action, horror]","[prison, android, spacecraft, space_marine, im...","[sigourney_weaver, charles_s._dutton, charles_...","[david_fincher, alex_thomson, elliot_goldentha..."
6316,39408,Left Behind III: World at War,Left Behind Collection,A year and a half ago the world was hit with t...,"[action, adventure, drama, fantasy, science_fi...",[],"[louis_gossett,_jr., kirk_cameron, brad_johnso...",[craig_r._baxley]


### 3.3. Cast and crew based recommendation

In [63]:
class CastAndCrewBasedRecommender(BinaryRecommender):
    # Recommender based on cast and crew

    def __init__(
        self,
        vectorizer,
        movies_data: pd.DataFrame = movies
    ):
        super().__init__(movies_data)
        
        vec = vectorizer()
        features_matrix = vec.fit_transform(
            (self.movies['cast'] + self.movies['crew']).apply(lambda x: ' '.join(x))
        )
        self.cosine_sim_matrix = cosine_similarity(features_matrix)

In [64]:
cbr = CastAndCrewBasedRecommender(CountVectorizer)
cbr.validate()

Recommendations for Toy Story:


,movie_id,title,collection,description,genres,keywords,cast,crew
2522,3114,Toy Story 2,Toy Story Collection,"Andy heads off to Cowboy Camp, leaving his toy...","[animation, comedy, family]","[museum, prosecution, identity_crisis, airplan...","[tom_hanks, tim_allen, joan_cusack, kelsey_gra...","[andrew_stanton, andrew_stanton, lee_unkrich, ..."
7621,78499,Toy Story 3,Toy Story Collection,"Woody, Buzz, and the rest of Andy's toys haven...","[animation, family, comedy]","[hostage, college, toy, barbie, animation, esc...","[tom_hanks, tim_allen, ned_beatty, joan_cusack...","[andrew_stanton, lee_unkrich, lee_unkrich, joh..."
8503,106022,Toy Story of Terror!,,What starts out as a fun road trip for the Toy...,"[animation, comedy, family]","[halloween, toys, short]","[tom_hanks, tim_allen, kristen_schaal, carl_we...","[angus_maclane, john_lasseter, andrew_stanton,..."
4650,6377,Finding Nemo,Finding Nemo Collection,"Nemo, an adventurous young clownfish, is unexp...","[animation, family]","[father_son_relationship, harbor, underwater, ...","[albert_brooks, ellen_degeneres, alexander_gou...","[andrew_stanton, andrew_stanton, andrew_stanto..."
1883,2355,A Bug's Life,,"On behalf of ""oppressed bugs everywhere,"" an i...","[adventure, animation, comedy, family]","[winter, fight, ant, invention, collector, ant...","[kevin_spacey, julia_louis-dreyfus, hayden_pan...","[andrew_stanton, andrew_stanton, andrew_stanto..."
6378,42191,Luxo Jr.,,A baby lamp finds a ball to play with and it's...,[animation],"[luxo_lamps, beach_ball, father_and_child, short]",[],"[john_lasseter, john_lasseter, john_lasseter]"
7037,60069,WALL·E,,WALL·E is the last robot left on an Earth that...,"[animation, family]",[romantic_comedy],"[ben_burtt, elissa_knight, jeff_garlin, fred_w...","[andrew_stanton, andrew_stanton, andrew_stanto..."
3825,4886,"Monsters, Inc.","Monsters, Inc. Collection","James Sullivan and Mike Wazowski are monsters,...","[animation, comedy, family]","[monster, infant, energy_supply, company, riva...","[john_goodman, billy_crystal, mary_gibbs, stev...","[darla_k._anderson, john_lasseter, andrew_stan..."
9182,157296,Finding Dory,Finding Nemo Collection,Dory is reunited with her friends Nemo and Mar...,"[adventure, animation, comedy, family]","[fish, amnesia, sequel, animation, talking_ani...","[ellen_degeneres, albert_brooks, hayden_rolenc...","[andrew_stanton, andrew_stanton, andrew_stanto..."
8139,95311,Presto,,Dignity. Poise. Mystery. We expect nothing les...,"[animation, family]","[stage, banjo, magic, bunny, rabbi, rope, resc...",[doug_sweetland],"[andrew_stanton, lee_unkrich, john_lasseter, h..."




Recommendations for Avatar:


,movie_id,title,collection,description,genres,keywords,cast,crew
1376,1721,Titanic,,"84 years later, a 101-year-old woman named Ros...","[drama, romance, thriller]","[shipwreck, iceberg, ship, panic, titanic, oce...","[kate_winslet, leonardo_dicaprio, frances_fish...","[mali_finn, james_horner, james_cameron, james..."
522,589,Terminator 2: Judgment Day,The Terminator Collection,Nearly 10 years have passed since Sarah Connor...,"[action, thriller, science_fiction]","[cyborg, shotgun, post-apocalyptic, dystopia, ...","[arnold_schwarzenegger, linda_hamilton, robert...","[dody_dorn, john_m._dwyer, gale_anne_hurd, mar..."
344,380,True Lies,,Harry Tasker is a secret agent for the United ...,"[action, thriller]","[spy, terrorist, florida, gun, kidnapping, hor...","[arnold_schwarzenegger, jamie_lee_curtis, tom_...","[mark_goldblatt, mali_finn, cindy_carr, james_..."
1011,1240,The Terminator,The Terminator Collection,"In the post-apocalyptic future, reigning tyran...","[action, thriller, science_fiction]","[saving_the_world, artificial_intelligence, re...","[arnold_schwarzenegger, michael_biehn, linda_h...","[gale_anne_hurd, gale_anne_hurd, mark_goldblat..."
922,1127,The Abyss,,A civilian oil rig crew is recruited to conduc...,"[adventure, action, thriller, science_fiction]","[ocean, sea, diving_suit, flying_saucer, nucle...","[ed_harris, mary_elizabeth_mastrantonio, micha...","[alan_silvestri, dody_dorn, gale_anne_hurd, ho..."
174,198,Strange Days,,Set in the year 1999 during the last days of t...,"[crime, drama, science_fiction, thriller]","[pornography, prostitute, rape, police_brutali...","[ralph_fiennes, angela_bassett, tom_sizemore, ...","[howard_a._smith, steven-charles_jaffe, graeme..."
4339,5779,Piranha Part Two: The Spawning,Piranha Collection,"A scuba diving instructor, her biochemist boyf...",[horror],"[beach, nudity, sequel, blood, piranha, explod...","[tricia_o'neil, steve_marachuk, lance_henrikse...","[james_cameron, ovidio_g._assonitis, jeff_sche..."
974,1200,Aliens,Alien Collection,When Ripley's lifepod is found by a salvage cr...,"[horror, action, thriller, science_fiction]","[android, extraterrestrial_technology, space_m...","[sigourney_weaver, michael_biehn, james_remar,...","[james_cameron, gale_anne_hurd, james_horner, ..."
3359,4198,Battle Beyond the Stars,,A young farmer assembles a band of diverse mer...,[science_fiction],"[clone, hitman, outer_space, robot, explosion]","[richard_thomas, robert_vaughn, john_saxon, ge...","[jimmy_t._murakami, john_sayles, james_horner,..."
8712,112852,Guardians of the Galaxy,Guardians of the Galaxy Collection,"Light years from Earth, 26 years after being a...","[action, science_fiction, adventure]","[marvel_comic, spaceship, space, outer_space, ...","[chris_pratt, zoe_saldana, dave_bautista, vin_...","[craig_wood, christopher_boyes, christopher_bo..."


In [65]:
cbr = CastAndCrewBasedRecommender(TfidfVectorizer)
cbr.validate()

Recommendations for Toy Story:


,movie_id,title,collection,description,genres,keywords,cast,crew
2522,3114,Toy Story 2,Toy Story Collection,"Andy heads off to Cowboy Camp, leaving his toy...","[animation, comedy, family]","[museum, prosecution, identity_crisis, airplan...","[tom_hanks, tim_allen, joan_cusack, kelsey_gra...","[andrew_stanton, andrew_stanton, lee_unkrich, ..."
7621,78499,Toy Story 3,Toy Story Collection,"Woody, Buzz, and the rest of Andy's toys haven...","[animation, family, comedy]","[hostage, college, toy, barbie, animation, esc...","[tom_hanks, tim_allen, ned_beatty, joan_cusack...","[andrew_stanton, lee_unkrich, lee_unkrich, joh..."
8503,106022,Toy Story of Terror!,,What starts out as a fun road trip for the Toy...,"[animation, comedy, family]","[halloween, toys, short]","[tom_hanks, tim_allen, kristen_schaal, carl_we...","[angus_maclane, john_lasseter, andrew_stanton,..."
4650,6377,Finding Nemo,Finding Nemo Collection,"Nemo, an adventurous young clownfish, is unexp...","[animation, family]","[father_son_relationship, harbor, underwater, ...","[albert_brooks, ellen_degeneres, alexander_gou...","[andrew_stanton, andrew_stanton, andrew_stanto..."
1883,2355,A Bug's Life,,"On behalf of ""oppressed bugs everywhere,"" an i...","[adventure, animation, comedy, family]","[winter, fight, ant, invention, collector, ant...","[kevin_spacey, julia_louis-dreyfus, hayden_pan...","[andrew_stanton, andrew_stanton, andrew_stanto..."
6378,42191,Luxo Jr.,,A baby lamp finds a ball to play with and it's...,[animation],"[luxo_lamps, beach_ball, father_and_child, short]",[],"[john_lasseter, john_lasseter, john_lasseter]"
7037,60069,WALL·E,,WALL·E is the last robot left on an Earth that...,"[animation, family]",[romantic_comedy],"[ben_burtt, elissa_knight, jeff_garlin, fred_w...","[andrew_stanton, andrew_stanton, andrew_stanto..."
3825,4886,"Monsters, Inc.","Monsters, Inc. Collection","James Sullivan and Mike Wazowski are monsters,...","[animation, comedy, family]","[monster, infant, energy_supply, company, riva...","[john_goodman, billy_crystal, mary_gibbs, stev...","[darla_k._anderson, john_lasseter, andrew_stan..."
9182,157296,Finding Dory,Finding Nemo Collection,Dory is reunited with her friends Nemo and Mar...,"[adventure, animation, comedy, family]","[fish, amnesia, sequel, animation, talking_ani...","[ellen_degeneres, albert_brooks, hayden_rolenc...","[andrew_stanton, andrew_stanton, andrew_stanto..."
7239,66934,Dr. Horrible's Sing-Along Blog,,"Dr. Horrible, an aspiring supervillain with hi...","[adventure, action, comedy, science_fiction, m...","[anti_hero, musical, parody, mad_scientist, su...","[neil_patrick_harris, nathan_fillion, felicia_...","[joss_whedon, joss_whedon, joss_whedon, jed_wh..."




Recommendations for Avatar:


,movie_id,title,collection,description,genres,keywords,cast,crew
1376,1721,Titanic,,"84 years later, a 101-year-old woman named Ros...","[drama, romance, thriller]","[shipwreck, iceberg, ship, panic, titanic, oce...","[kate_winslet, leonardo_dicaprio, frances_fish...","[mali_finn, james_horner, james_cameron, james..."
522,589,Terminator 2: Judgment Day,The Terminator Collection,Nearly 10 years have passed since Sarah Connor...,"[action, thriller, science_fiction]","[cyborg, shotgun, post-apocalyptic, dystopia, ...","[arnold_schwarzenegger, linda_hamilton, robert...","[dody_dorn, john_m._dwyer, gale_anne_hurd, mar..."
344,380,True Lies,,Harry Tasker is a secret agent for the United ...,"[action, thriller]","[spy, terrorist, florida, gun, kidnapping, hor...","[arnold_schwarzenegger, jamie_lee_curtis, tom_...","[mark_goldblatt, mali_finn, cindy_carr, james_..."
1011,1240,The Terminator,The Terminator Collection,"In the post-apocalyptic future, reigning tyran...","[action, thriller, science_fiction]","[saving_the_world, artificial_intelligence, re...","[arnold_schwarzenegger, michael_biehn, linda_h...","[gale_anne_hurd, gale_anne_hurd, mark_goldblat..."
922,1127,The Abyss,,A civilian oil rig crew is recruited to conduc...,"[adventure, action, thriller, science_fiction]","[ocean, sea, diving_suit, flying_saucer, nucle...","[ed_harris, mary_elizabeth_mastrantonio, micha...","[alan_silvestri, dody_dorn, gale_anne_hurd, ho..."
174,198,Strange Days,,Set in the year 1999 during the last days of t...,"[crime, drama, science_fiction, thriller]","[pornography, prostitute, rape, police_brutali...","[ralph_fiennes, angela_bassett, tom_sizemore, ...","[howard_a._smith, steven-charles_jaffe, graeme..."
4339,5779,Piranha Part Two: The Spawning,Piranha Collection,"A scuba diving instructor, her biochemist boyf...",[horror],"[beach, nudity, sequel, blood, piranha, explod...","[tricia_o'neil, steve_marachuk, lance_henrikse...","[james_cameron, ovidio_g._assonitis, jeff_sche..."
974,1200,Aliens,Alien Collection,When Ripley's lifepod is found by a salvage cr...,"[horror, action, thriller, science_fiction]","[android, extraterrestrial_technology, space_m...","[sigourney_weaver, michael_biehn, james_remar,...","[james_cameron, gale_anne_hurd, james_horner, ..."
8712,112852,Guardians of the Galaxy,Guardians of the Galaxy Collection,"Light years from Earth, 26 years after being a...","[action, science_fiction, adventure]","[marvel_comic, spaceship, space, outer_space, ...","[chris_pratt, zoe_saldana, dave_bautista, vin_...","[craig_wood, christopher_boyes, christopher_bo..."
8868,122892,Avengers: Age of Ultron,The Avengers Collection,When Tony Stark tries to jumpstart a dormant p...,"[action, adventure, science_fiction]","[marvel_comic, sequel, superhero, based_on_com...","[robert_downey_jr., chris_hemsworth, mark_ruff...","[danny_elfman, christopher_boyes, christopher_..."


### 3.4. All metadata based recommendation

In [66]:
class AllMetadataBasedRecommender(BinaryRecommender):
    # Recommender based on all previous used metadata

    def __init__(
        self,
        vectorizer,
        movies_data: pd.DataFrame = movies
    ):
        super().__init__(movies_data)
        
        vec = vectorizer()
        
        genres_keywords = (self.movies['genres'] + self.movies['keywords']).apply(lambda x: ' '.join(x))
        cast_crew = (self.movies['cast'] + self.movies['crew']).apply(lambda x: ' '.join(x))
        data = (self.movies['description'] + ' ' + genres_keywords + ' ' + cast_crew)

        features_matrix = vec.fit_transform(data)
        self.cosine_sim_matrix = cosine_similarity(features_matrix)

In [67]:
cbr = AllMetadataBasedRecommender(CountVectorizer)
cbr.validate()

Recommendations for Toy Story:


,movie_id,title,collection,description,genres,keywords,cast,crew
2522,3114,Toy Story 2,Toy Story Collection,"Andy heads off to Cowboy Camp, leaving his toy...","[animation, comedy, family]","[museum, prosecution, identity_crisis, airplan...","[tom_hanks, tim_allen, joan_cusack, kelsey_gra...","[andrew_stanton, andrew_stanton, lee_unkrich, ..."
7621,78499,Toy Story 3,Toy Story Collection,"Woody, Buzz, and the rest of Andy's toys haven...","[animation, family, comedy]","[hostage, college, toy, barbie, animation, esc...","[tom_hanks, tim_allen, ned_beatty, joan_cusack...","[andrew_stanton, lee_unkrich, lee_unkrich, joh..."
4306,5690,Grave of the Fireflies,,"In the latter part of World War II, a boy and ...","[animation, drama, war]","[japan, brother_sister_relationship, hunger, w...","[tsutomu_tatsumi, ayano_shiraishi, yoshiko_shi...","[isao_takahata, isao_takahata, takeshi_seyama,..."
7188,65133,Blackadder Back & Forth,,It's Millennium Eve and Blackadder is hosting ...,"[comedy, history]",[],"[rowan_atkinson, tony_robinson, stephen_fry, h...","[richard_curtis, paul_weiland, ben_elton]"
1680,2103,Tall Tale,,A young boy draws on the inspiration of legend...,[adventure],[],"[patrick_swayze, oliver_platt, roger_aaron_bro...","[jeremiah_s._chechik, steve_bloom, robert_roda..."
1960,2434,Down in the Delta,,Rosa Lynn sends her druggie daughter Loretta a...,[drama],[woman_director],"[alfre_woodard, al_freeman,_jr., esther_rolle,...","[maya_angelou, myron_goble]"
7221,66246,Uuno Turhapuro,Uuno Turhapuro,Uuno goes with his wife to spend summer holida...,[comedy],[],"[vesa-matti_loiri, marjatta_raita, spede_pasan...","[ere_kokkonen, spede_pasanen]"
2389,2956,Someone to Watch Over Me,,"In Queens, Mike Keegan is celebrating with his...","[crime, drama, romance, thriller]","[new_york, detective, marriage, murder, social...","[tom_berenger, mimi_rogers, lorraine_bracco, j...","[colleen_atwood, ridley_scott, howard_franklin]"
5407,8199,Ugetsu,,"In the civil wars of 16th century Japan, two a...","[fantasy, drama, mystery]","[japan, based_on_novel, samurai, greed, bigamy...","[masayuki_mori, machiko_kyô, kinuyo_tanaka, ei...","[masaichi_nagata, fumio_hayasaka, kazuo_miyaga..."
2876,3578,Gladiator,,"In the year 180, the death of emperor Marcus A...","[action, drama, adventure]","[rome, gladiator, arena, senate, roman_empire,...","[russell_crowe, joaquin_phoenix, connie_nielse...","[ridley_scott, david_franzoni, branko_lustig, ..."




Recommendations for Avatar:


,movie_id,title,collection,description,genres,keywords,cast,crew
6915,56801,Aliens vs Predator: Requiem,AVP Collection,"A sequel to 2004's Alien vs. Predator, the ico...","[fantasy, action, science_fiction, thriller, h...","[predator, national_guard, hybrid, alien, morg...","[steven_pasquale, reiko_aylesworth, john_ortiz...","[david_giler, jim_thomas, john_thomas, walter_..."
522,589,Terminator 2: Judgment Day,The Terminator Collection,Nearly 10 years have passed since Sarah Connor...,"[action, thriller, science_fiction]","[cyborg, shotgun, post-apocalyptic, dystopia, ...","[arnold_schwarzenegger, linda_hamilton, robert...","[dody_dorn, john_m._dwyer, gale_anne_hurd, mar..."
1376,1721,Titanic,,"84 years later, a 101-year-old woman named Ros...","[drama, romance, thriller]","[shipwreck, iceberg, ship, panic, titanic, oce...","[kate_winslet, leonardo_dicaprio, frances_fish...","[mali_finn, james_horner, james_cameron, james..."
8739,114028,Pride,,Based on a trues story. In the summer of 1984 ...,"[drama, comedy]",[lgbt],"[ben_schnetzer, bill_nighy, imelda_staunton, d...","[matthew_warchus, stephen_beresford, david_liv..."
6955,58156,Semi-Pro,,"Jackie Moon is the owner, promoter, coach, and...",[comedy],"[sport, basketball, flint_michigan, nba, merge...","[will_ferrell, maura_tierney, woody_harrelson,...","[scot_armstrong, kent_alterman, brian_avery]"
3457,4337,The Sand Pebbles,,Engineer Jake Holman arrives aboard the gunboa...,"[action, adventure, drama, war, romance]","[missionary, china, u.s._navy, chinese, battle...","[steve_mcqueen, richard_attenborough, richard_...","[robert_wise, robert_wise, boris_leven, jerry_..."
8618,110102,Captain America: The Winter Soldier,Captain America Collection,After the cataclysmic events in New York with ...,"[action, adventure, science_fiction]","[washington_d.c., future, shield, marvel_comic...","[chris_evans, samuel_l._jackson, scarlett_joha...","[sarah_finn, beat_frutiger, christopher_markus..."
5419,8257,Battleground,,The film concentrates on the camaraderie and t...,"[action, adventure, drama, war]",[world_war_ii],"[van_johnson, john_hodiak, ricardo_montalban, ...","[cedric_gibbons, john_d._dunning, william_a._w..."
1133,1376,Star Trek IV: The Voyage Home,Star Trek: The Original Series Collection,Fugitives of the Federation for their daring r...,"[science_fiction, adventure]","[saving_the_world, san_francisco, uss_enterpri...","[william_shatner, leonard_nimoy, deforest_kell...","[leonard_nimoy, harve_bennett, steve_meerson, ..."
6768,53138,The Librarian: Return to King Solomon's Mines,The Librarian Collection,"After retrieving the Crystal Skull in Utah, Fl...","[fantasy, action, adventure]","[incantation, treasure_hunt, treasure_map, sci...","[noah_wyle, gabrielle_anwar, bob_newhart, jane...","[jonathan_frakes, david_n._titcher, marco_schn..."


In [68]:
cbr = AllMetadataBasedRecommender(TfidfVectorizer)
cbr.validate()

Recommendations for Toy Story:


,movie_id,title,collection,description,genres,keywords,cast,crew
2522,3114,Toy Story 2,Toy Story Collection,"Andy heads off to Cowboy Camp, leaving his toy...","[animation, comedy, family]","[museum, prosecution, identity_crisis, airplan...","[tom_hanks, tim_allen, joan_cusack, kelsey_gra...","[andrew_stanton, andrew_stanton, lee_unkrich, ..."
7621,78499,Toy Story 3,Toy Story Collection,"Woody, Buzz, and the rest of Andy's toys haven...","[animation, family, comedy]","[hostage, college, toy, barbie, animation, esc...","[tom_hanks, tim_allen, ned_beatty, joan_cusack...","[andrew_stanton, lee_unkrich, lee_unkrich, joh..."
8503,106022,Toy Story of Terror!,,What starts out as a fun road trip for the Toy...,"[animation, comedy, family]","[halloween, toys, short]","[tom_hanks, tim_allen, kristen_schaal, carl_we...","[angus_maclane, john_lasseter, andrew_stanton,..."
6259,35836,The 40 Year Old Virgin,,Andy Stitzer has a pleasant life with a nice a...,"[comedy, romance]","[first_time, virgin]","[steve_carell, catherine_keener, paul_rudd, ro...","[steve_carell, zachary_woodlee, judd_apatow, j..."
4749,6584,"What's Up, Tiger Lily?",,"In comic Woody Allen's film debut, he took the...","[action, comedy]",[spoof],"[tatsuya_mihashi, akiko_wakabayashi, mie_hama,...","[woody_allen, senkichi_taniguchi]"
3825,4886,"Monsters, Inc.","Monsters, Inc. Collection","James Sullivan and Mike Wazowski are monsters,...","[animation, comedy, family]","[monster, infant, energy_supply, company, riva...","[john_goodman, billy_crystal, mary_gibbs, stev...","[darla_k._anderson, john_lasseter, andrew_stan..."
910,1103,Rebel Without a Cause,,"After moving to a new town, troublemaking teen...",[drama],"[individual, rebel, street_gang, car_race, par...","[james_dean, natalie_wood, sal_mineo, ann_dora...","[nicholas_ray, david_weisbart, nicholas_ray, i..."
6700,51174,Factory Girl,,"In the mid-1960s, wealthy debutant Edie Sedgwi...",[drama],"[new_york, alcohol, sex, infidelity, nudity, a...","[sienna_miller, guy_pearce, hayden_christensen...","[ed_shearmur, john_dunn, george_hickenlooper, ..."
1619,2041,Condorman,,Comic artist and writer Woody performs a simpl...,"[comedy, action, adventure]",[mission],"[michael_crawford, oliver_reed, barbara_carrer...","[charles_jarrott, marc_stirdivant, henry_manci..."
1883,2355,A Bug's Life,,"On behalf of ""oppressed bugs everywhere,"" an i...","[adventure, animation, comedy, family]","[winter, fight, ant, invention, collector, ant...","[kevin_spacey, julia_louis-dreyfus, hayden_pan...","[andrew_stanton, andrew_stanton, andrew_stanto..."




Recommendations for Avatar:


,movie_id,title,collection,description,genres,keywords,cast,crew
522,589,Terminator 2: Judgment Day,The Terminator Collection,Nearly 10 years have passed since Sarah Connor...,"[action, thriller, science_fiction]","[cyborg, shotgun, post-apocalyptic, dystopia, ...","[arnold_schwarzenegger, linda_hamilton, robert...","[dody_dorn, john_m._dwyer, gale_anne_hurd, mar..."
344,380,True Lies,,Harry Tasker is a secret agent for the United ...,"[action, thriller]","[spy, terrorist, florida, gun, kidnapping, hor...","[arnold_schwarzenegger, jamie_lee_curtis, tom_...","[mark_goldblatt, mali_finn, cindy_carr, james_..."
1011,1240,The Terminator,The Terminator Collection,"In the post-apocalyptic future, reigning tyran...","[action, thriller, science_fiction]","[saving_the_world, artificial_intelligence, re...","[arnold_schwarzenegger, michael_biehn, linda_h...","[gale_anne_hurd, gale_anne_hurd, mark_goldblat..."
1376,1721,Titanic,,"84 years later, a 101-year-old woman named Ros...","[drama, romance, thriller]","[shipwreck, iceberg, ship, panic, titanic, oce...","[kate_winslet, leonardo_dicaprio, frances_fish...","[mali_finn, james_horner, james_cameron, james..."
3013,3745,Titan A.E.,,A young man finds out that he holds the key to...,"[animation, action, science_fiction, family, a...","[monster, galaxy, dystopia, space, alien, anim...","[matt_damon, bill_pullman, drew_barrymore, joh...","[christopher_boyes, christopher_boyes, john_au..."
974,1200,Aliens,Alien Collection,When Ripley's lifepod is found by a salvage cr...,"[horror, action, thriller, science_fiction]","[android, extraterrestrial_technology, space_m...","[sigourney_weaver, michael_biehn, james_remar,...","[james_cameron, gale_anne_hurd, james_horner, ..."
922,1127,The Abyss,,A civilian oil rig crew is recruited to conduc...,"[adventure, action, thriller, science_fiction]","[ocean, sea, diving_suit, flying_saucer, nucle...","[ed_harris, mary_elizabeth_mastrantonio, micha...","[alan_silvestri, dody_dorn, gale_anne_hurd, ho..."
174,198,Strange Days,,Set in the year 1999 during the last days of t...,"[crime, drama, science_fiction, thriller]","[pornography, prostitute, rape, police_brutali...","[ralph_fiennes, angela_bassett, tom_sizemore, ...","[howard_a._smith, steven-charles_jaffe, graeme..."
8712,112852,Guardians of the Galaxy,Guardians of the Galaxy Collection,"Light years from Earth, 26 years after being a...","[action, science_fiction, adventure]","[marvel_comic, spaceship, space, outer_space, ...","[chris_pratt, zoe_saldana, dave_bautista, vin_...","[craig_wood, christopher_boyes, christopher_bo..."
4339,5779,Piranha Part Two: The Spawning,Piranha Collection,"A scuba diving instructor, her biochemist boyf...",[horror],"[beach, nudity, sequel, blood, piranha, explod...","[tricia_o'neil, steve_marachuk, lance_henrikse...","[james_cameron, ovidio_g._assonitis, jeff_sche..."


### 3.5. Genres, keywords, cast and crew based recommendation

In [69]:
class NoDescriptionBasedRecommender(BinaryRecommender):
    # Recommender based on genres, keywords, cast and crew

    def __init__(
        self,
        vectorizer,
        movies_data: pd.DataFrame = movies
    ):
        super().__init__(movies_data)
        
        vec = vectorizer()
        
        genres_keywords = (self.movies['genres'] + self.movies['keywords']).apply(lambda x: ' '.join(x))
        cast_crew = (self.movies['cast'] + self.movies['crew']).apply(lambda x: ' '.join(x))
        data = genres_keywords + ' ' + cast_crew

        features_matrix = vec.fit_transform(data)
        self.cosine_sim_matrix = cosine_similarity(features_matrix)

In [70]:
cbr = NoDescriptionBasedRecommender(CountVectorizer)
cbr.validate()

Recommendations for Toy Story:


,movie_id,title,collection,description,genres,keywords,cast,crew
2522,3114,Toy Story 2,Toy Story Collection,"Andy heads off to Cowboy Camp, leaving his toy...","[animation, comedy, family]","[museum, prosecution, identity_crisis, airplan...","[tom_hanks, tim_allen, joan_cusack, kelsey_gra...","[andrew_stanton, andrew_stanton, lee_unkrich, ..."
7621,78499,Toy Story 3,Toy Story Collection,"Woody, Buzz, and the rest of Andy's toys haven...","[animation, family, comedy]","[hostage, college, toy, barbie, animation, esc...","[tom_hanks, tim_allen, ned_beatty, joan_cusack...","[andrew_stanton, lee_unkrich, lee_unkrich, joh..."
8503,106022,Toy Story of Terror!,,What starts out as a fun road trip for the Toy...,"[animation, comedy, family]","[halloween, toys, short]","[tom_hanks, tim_allen, kristen_schaal, carl_we...","[angus_maclane, john_lasseter, andrew_stanton,..."
2751,3429,Creature Comforts,,Interviews with the public are used to put wor...,"[animation, comedy, family]",[animation],[],[nick_park]
3825,4886,"Monsters, Inc.","Monsters, Inc. Collection","James Sullivan and Mike Wazowski are monsters,...","[animation, comedy, family]","[monster, infant, energy_supply, company, riva...","[john_goodman, billy_crystal, mary_gibbs, stev...","[darla_k._anderson, john_lasseter, andrew_stan..."
7037,60069,WALL·E,,WALL·E is the last robot left on an Earth that...,"[animation, family]",[romantic_comedy],"[ben_burtt, elissa_knight, jeff_garlin, fred_w...","[andrew_stanton, andrew_stanton, andrew_stanto..."
1883,2355,A Bug's Life,,"On behalf of ""oppressed bugs everywhere,"" an i...","[adventure, animation, comedy, family]","[winter, fight, ant, invention, collector, ant...","[kevin_spacey, julia_louis-dreyfus, hayden_pan...","[andrew_stanton, andrew_stanton, andrew_stanto..."
4650,6377,Finding Nemo,Finding Nemo Collection,"Nemo, an adventurous young clownfish, is unexp...","[animation, family]","[father_son_relationship, harbor, underwater, ...","[albert_brooks, ellen_degeneres, alexander_gou...","[andrew_stanton, andrew_stanton, andrew_stanto..."
9182,157296,Finding Dory,Finding Nemo Collection,Dory is reunited with her friends Nemo and Mar...,"[adventure, animation, comedy, family]","[fish, amnesia, sequel, animation, talking_ani...","[ellen_degeneres, albert_brooks, hayden_rolenc...","[andrew_stanton, andrew_stanton, andrew_stanto..."
8992,134853,Inside Out,,"Growing up can be a bumpy road, and it's no ex...","[drama, comedy, animation, family]","[dream, cartoon, imaginary_friend, animation, ...","[amy_poehler, phyllis_smith, richard_kind, bil...","[pete_docter, jonas_rivera, pete_docter, john_..."




Recommendations for Avatar:


,movie_id,title,collection,description,genres,keywords,cast,crew
522,589,Terminator 2: Judgment Day,The Terminator Collection,Nearly 10 years have passed since Sarah Connor...,"[action, thriller, science_fiction]","[cyborg, shotgun, post-apocalyptic, dystopia, ...","[arnold_schwarzenegger, linda_hamilton, robert...","[dody_dorn, john_m._dwyer, gale_anne_hurd, mar..."
1376,1721,Titanic,,"84 years later, a 101-year-old woman named Ros...","[drama, romance, thriller]","[shipwreck, iceberg, ship, panic, titanic, oce...","[kate_winslet, leonardo_dicaprio, frances_fish...","[mali_finn, james_horner, james_cameron, james..."
344,380,True Lies,,Harry Tasker is a secret agent for the United ...,"[action, thriller]","[spy, terrorist, florida, gun, kidnapping, hor...","[arnold_schwarzenegger, jamie_lee_curtis, tom_...","[mark_goldblatt, mali_finn, cindy_carr, james_..."
1011,1240,The Terminator,The Terminator Collection,"In the post-apocalyptic future, reigning tyran...","[action, thriller, science_fiction]","[saving_the_world, artificial_intelligence, re...","[arnold_schwarzenegger, michael_biehn, linda_h...","[gale_anne_hurd, gale_anne_hurd, mark_goldblat..."
922,1127,The Abyss,,A civilian oil rig crew is recruited to conduc...,"[adventure, action, thriller, science_fiction]","[ocean, sea, diving_suit, flying_saucer, nucle...","[ed_harris, mary_elizabeth_mastrantonio, micha...","[alan_silvestri, dody_dorn, gale_anne_hurd, ho..."
974,1200,Aliens,Alien Collection,When Ripley's lifepod is found by a salvage cr...,"[horror, action, thriller, science_fiction]","[android, extraterrestrial_technology, space_m...","[sigourney_weaver, michael_biehn, james_remar,...","[james_cameron, gale_anne_hurd, james_horner, ..."
174,198,Strange Days,,Set in the year 1999 during the last days of t...,"[crime, drama, science_fiction, thriller]","[pornography, prostitute, rape, police_brutali...","[ralph_fiennes, angela_bassett, tom_sizemore, ...","[howard_a._smith, steven-charles_jaffe, graeme..."
8712,112852,Guardians of the Galaxy,Guardians of the Galaxy Collection,"Light years from Earth, 26 years after being a...","[action, science_fiction, adventure]","[marvel_comic, spaceship, space, outer_space, ...","[chris_pratt, zoe_saldana, dave_bautista, vin_...","[craig_wood, christopher_boyes, christopher_bo..."
3013,3745,Titan A.E.,,A young man finds out that he holds the key to...,"[animation, action, science_fiction, family, a...","[monster, galaxy, dystopia, space, alien, anim...","[matt_damon, bill_pullman, drew_barrymore, joh...","[christopher_boyes, christopher_boyes, john_au..."
4329,5746,Galaxy of Terror,,As a lone spaceship proceeds on its long voyag...,"[action, adventure, horror, science_fiction]","[monster, alien, astronaut, alien_possession, ...","[edward_albert, erin_moran, ray_walston, berna...","[bruce_d._clark, rupert_harvey, don_keith_oppe..."


In [71]:
cbr = NoDescriptionBasedRecommender(TfidfVectorizer)
cbr.validate()

Recommendations for Toy Story:


,movie_id,title,collection,description,genres,keywords,cast,crew
2522,3114,Toy Story 2,Toy Story Collection,"Andy heads off to Cowboy Camp, leaving his toy...","[animation, comedy, family]","[museum, prosecution, identity_crisis, airplan...","[tom_hanks, tim_allen, joan_cusack, kelsey_gra...","[andrew_stanton, andrew_stanton, lee_unkrich, ..."
7621,78499,Toy Story 3,Toy Story Collection,"Woody, Buzz, and the rest of Andy's toys haven...","[animation, family, comedy]","[hostage, college, toy, barbie, animation, esc...","[tom_hanks, tim_allen, ned_beatty, joan_cusack...","[andrew_stanton, lee_unkrich, lee_unkrich, joh..."
8503,106022,Toy Story of Terror!,,What starts out as a fun road trip for the Toy...,"[animation, comedy, family]","[halloween, toys, short]","[tom_hanks, tim_allen, kristen_schaal, carl_we...","[angus_maclane, john_lasseter, andrew_stanton,..."
4650,6377,Finding Nemo,Finding Nemo Collection,"Nemo, an adventurous young clownfish, is unexp...","[animation, family]","[father_son_relationship, harbor, underwater, ...","[albert_brooks, ellen_degeneres, alexander_gou...","[andrew_stanton, andrew_stanton, andrew_stanto..."
7037,60069,WALL·E,,WALL·E is the last robot left on an Earth that...,"[animation, family]",[romantic_comedy],"[ben_burtt, elissa_knight, jeff_garlin, fred_w...","[andrew_stanton, andrew_stanton, andrew_stanto..."
1883,2355,A Bug's Life,,"On behalf of ""oppressed bugs everywhere,"" an i...","[adventure, animation, comedy, family]","[winter, fight, ant, invention, collector, ant...","[kevin_spacey, julia_louis-dreyfus, hayden_pan...","[andrew_stanton, andrew_stanton, andrew_stanto..."
3825,4886,"Monsters, Inc.","Monsters, Inc. Collection","James Sullivan and Mike Wazowski are monsters,...","[animation, comedy, family]","[monster, infant, energy_supply, company, riva...","[john_goodman, billy_crystal, mary_gibbs, stev...","[darla_k._anderson, john_lasseter, andrew_stan..."
9182,157296,Finding Dory,Finding Nemo Collection,Dory is reunited with her friends Nemo and Mar...,"[adventure, animation, comedy, family]","[fish, amnesia, sequel, animation, talking_ani...","[ellen_degeneres, albert_brooks, hayden_rolenc...","[andrew_stanton, andrew_stanton, andrew_stanto..."
6378,42191,Luxo Jr.,,A baby lamp finds a ball to play with and it's...,[animation],"[luxo_lamps, beach_ball, father_and_child, short]",[],"[john_lasseter, john_lasseter, john_lasseter]"
8142,95377,One Man Band,,With one coin to make a wish at the piazza fou...,"[animation, family]","[fountain, rivalry, coin, bag_of_money, gold_c...",[],"[john_lasseter, marcia_gwendolyn_jones, angus_..."




Recommendations for Avatar:


,movie_id,title,collection,description,genres,keywords,cast,crew
1376,1721,Titanic,,"84 years later, a 101-year-old woman named Ros...","[drama, romance, thriller]","[shipwreck, iceberg, ship, panic, titanic, oce...","[kate_winslet, leonardo_dicaprio, frances_fish...","[mali_finn, james_horner, james_cameron, james..."
522,589,Terminator 2: Judgment Day,The Terminator Collection,Nearly 10 years have passed since Sarah Connor...,"[action, thriller, science_fiction]","[cyborg, shotgun, post-apocalyptic, dystopia, ...","[arnold_schwarzenegger, linda_hamilton, robert...","[dody_dorn, john_m._dwyer, gale_anne_hurd, mar..."
344,380,True Lies,,Harry Tasker is a secret agent for the United ...,"[action, thriller]","[spy, terrorist, florida, gun, kidnapping, hor...","[arnold_schwarzenegger, jamie_lee_curtis, tom_...","[mark_goldblatt, mali_finn, cindy_carr, james_..."
1011,1240,The Terminator,The Terminator Collection,"In the post-apocalyptic future, reigning tyran...","[action, thriller, science_fiction]","[saving_the_world, artificial_intelligence, re...","[arnold_schwarzenegger, michael_biehn, linda_h...","[gale_anne_hurd, gale_anne_hurd, mark_goldblat..."
174,198,Strange Days,,Set in the year 1999 during the last days of t...,"[crime, drama, science_fiction, thriller]","[pornography, prostitute, rape, police_brutali...","[ralph_fiennes, angela_bassett, tom_sizemore, ...","[howard_a._smith, steven-charles_jaffe, graeme..."
922,1127,The Abyss,,A civilian oil rig crew is recruited to conduc...,"[adventure, action, thriller, science_fiction]","[ocean, sea, diving_suit, flying_saucer, nucle...","[ed_harris, mary_elizabeth_mastrantonio, micha...","[alan_silvestri, dody_dorn, gale_anne_hurd, ho..."
974,1200,Aliens,Alien Collection,When Ripley's lifepod is found by a salvage cr...,"[horror, action, thriller, science_fiction]","[android, extraterrestrial_technology, space_m...","[sigourney_weaver, michael_biehn, james_remar,...","[james_cameron, gale_anne_hurd, james_horner, ..."
8712,112852,Guardians of the Galaxy,Guardians of the Galaxy Collection,"Light years from Earth, 26 years after being a...","[action, science_fiction, adventure]","[marvel_comic, spaceship, space, outer_space, ...","[chris_pratt, zoe_saldana, dave_bautista, vin_...","[craig_wood, christopher_boyes, christopher_bo..."
3013,3745,Titan A.E.,,A young man finds out that he holds the key to...,"[animation, action, science_fiction, family, a...","[monster, galaxy, dystopia, space, alien, anim...","[matt_damon, bill_pullman, drew_barrymore, joh...","[christopher_boyes, christopher_boyes, john_au..."
4339,5779,Piranha Part Two: The Spawning,Piranha Collection,"A scuba diving instructor, her biochemist boyf...",[horror],"[beach, nudity, sequel, blood, piranha, explod...","[tricia_o'neil, steve_marachuk, lance_henrikse...","[james_cameron, ovidio_g._assonitis, jeff_sche..."


This project is inspired by:  
https://github.com/sanchitbhasin/Content-Based-Movie-Recommendation-System/blob/master/movie%20recommendation%20system.ipynb  
https://github.com/vjvishaljha/Recommender_Sys/blob/master/Recommender%20System.ipynb
